## This is the midterm submission for Charles Benello on 10/20


I used chatgpt

In [12]:
# imports

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

import math
import datetime
from typing import Union, List


In [ ]:
# get data
file_path_1 = "data/midterm_1_stock_returns.xlsx"
file_path_2 = "data/midterm_1_fund_returns.xlsx"

sheet_name_1, sheet_name_2, sheet_name_3 = "Sheet1", "Sheet2", "Sheet3"  # fill these

df1 = pd.read_excel(file_path_1, sheet_name=sheet_name_1, index_col=0)
df2 = pd.read_excel(file_path_2, sheet_name=sheet_name_2, index_col=0)
# df3 = pd.read_excel(file_path, sheet_name=sheet_name_3, index_col=0)

df1.index = pd.to_datetime(df1.index)
df2.index = pd.to_datetime(df2.index)
df3.index = pd.to_datetime(df3.index)

x, y, z = df1.copy(), df2.copy(), df3.copy()

ValueError: Worksheet named 'Sheet1' not found

# Helpers

* Hw1

In [17]:
def calc_tangency_weights(
    returns: pd.DataFrame,
    cov_mat: str = 1,
    return_graphic: bool = False,
    return_port_ret: bool = False,
    target_ret_rescale_weights: Union[None, float] = None,
    annual_factor: int = 12,
    name: str = 'Tangency',
    expected_returns: Union[pd.Series, pd.DataFrame] = None,
    expected_returns_already_annualized: bool = False
):
    """
    Calculates tangency portfolio weights based on the covariance matrix of returns.

    Parameters:
    returns (pd.DataFrame): Time series of returns.
    cov_mat (str, default=1): Covariance matrix for calculating tangency weights.
    return_graphic (bool, default=False): If True, plots the tangency weights.
    return_port_ret (bool, default=False): If True, returns the portfolio returns.
    target_ret_rescale_weights (float or None, default=None): Target return for rescaling weights.
    annual_factor (int, default=12): Factor for annualizing returns.
    name (str, default='Tangency'): Name for labeling the weights and portfolio.

    Returns:
    pd.DataFrame or pd.Series: Tangency portfolio weights or portfolio returns if `return_port_ret` is True.
    """
    returns = returns.copy()
    
    if 'date' in returns.columns.str.lower():
        returns = returns.rename({'Date': 'date'}, axis=1)
        returns = returns.set_index('date')
    returns.index.name = 'date'

    if cov_mat == 1:
        cov_inv = np.linalg.inv((returns.cov() * annual_factor))
    else:
        cov = returns.cov()
        covmat_diag = np.diag(np.diag((cov)))
        covmat = cov_mat * cov + (1 - cov_mat) * covmat_diag
        cov_inv = np.linalg.pinv((covmat * annual_factor))  
        
    ones = np.ones(returns.columns.shape) 
    if expected_returns is not None:
        mu = expected_returns
        if not expected_returns_already_annualized:
            mu *= annual_factor
    else:
        mu = returns.mean() * annual_factor
    scaling = 1 / (np.transpose(ones) @ cov_inv @ mu)
    tangent_return = scaling * (cov_inv @ mu)
    tangency_wts = pd.DataFrame(
        index=returns.columns,
        data=tangent_return,
        columns=[f'{name} Weights']
    )
    port_returns = returns @ tangency_wts.rename({f'{name} Weights': f'{name} Portfolio'}, axis=1)

    if return_graphic:
        tangency_wts.plot(kind='bar', title=f'{name} Weights')

    if isinstance(target_ret_rescale_weights, (float, int)):
        scaler = target_ret_rescale_weights / port_returns[f'{name} Portfolio'].mean()
        tangency_wts[[f'{name} Weights']] *= scaler
        port_returns *= scaler
        tangency_wts = tangency_wts.rename(
            {f'{name} Weights': f'{name} Weights Rescaled Target {target_ret_rescale_weights:.2%}'},
            axis=1
        )
        port_returns = port_returns.rename(
            {f'{name} Portfolio': f'{name} Portfolio Rescaled Target {target_ret_rescale_weights:.2%}'},
            axis=1
        )

    if cov_mat != 1:
        port_returns = port_returns.rename(columns=lambda c: c.replace('Tangency', f'Tangency Regularized {cov_mat:.2f}'))
        tangency_wts = tangency_wts.rename(columns=lambda c: c.replace('Tangency', f'Tangency Regularized {cov_mat:.2f}'))
        
    if return_port_ret:
        return port_returns
    return tangency_wts

def filter_columns_and_indexes(
    df: pd.DataFrame,
    keep_columns: Union[list, str],
    drop_columns: Union[list, str],
    keep_indexes: Union[list, str],
    drop_indexes: Union[list, str],
    drop_before_keep: bool = False
):
    """
    Filters a DataFrame based on specified columns and indexes.

    Parameters:
    df (pd.DataFrame): DataFrame to be filtered.
    keep_columns (list or str): Columns to keep in the DataFrame.
    drop_columns (list or str): Columns to drop from the DataFrame.
    keep_indexes (list or str): Indexes to keep in the DataFrame.
    drop_indexes (list or str): Indexes to drop from the DataFrame.
    drop_before_keep (bool, default=False): Whether to drop specified columns/indexes before keeping.

    Returns:
    pd.DataFrame: The filtered DataFrame.
    """
    if not isinstance(df, (pd.DataFrame, pd.Series)):
        return df
    df = df.copy()
    # Columns
    if keep_columns is not None:
        keep_columns = "(?i)" + "|".join(keep_columns) if isinstance(keep_columns, list) else "(?i)" + keep_columns
    else:
        keep_columns = None
    if drop_columns is not None:
        drop_columns = "(?i)" + "|".join(drop_columns) if isinstance(drop_columns, list) else "(?i)" + drop_columns
    else:
        drop_columns = None
    if not drop_before_keep:
        if keep_columns is not None:
            df = df.filter(regex=keep_columns)
    if drop_columns is not None:
        df = df.drop(columns=df.filter(regex=drop_columns).columns)
    if drop_before_keep:
        if keep_columns is not None:
            df = df.filter(regex=keep_columns)
    # Indexes
    if keep_indexes is not None:
        keep_indexes = "(?i)" + "|".join(keep_indexes) if isinstance(keep_indexes, list) else "(?i)" + keep_indexes
    else:
        keep_indexes = None
    if drop_indexes is not None:
        drop_indexes = "(?i)" + "|".join(drop_indexes) if isinstance(drop_indexes, list) else "(?i)" + drop_indexes
    else:
        drop_indexes = None
    if not drop_before_keep:
        if keep_indexes is not None:
            df = df.filter(regex=keep_indexes, axis=0)
    if drop_indexes is not None:
        df = df.drop(index=df.filter(regex=drop_indexes, axis=0).index)
    if drop_before_keep:
        if keep_indexes is not None:
            df = df.filter(regex=keep_indexes, axis=0)
    return df

def calc_summary_statistics(
    returns: Union[pd.DataFrame, List],
    annual_factor: int = None,
    provided_excess_returns: bool = None,
    rf: Union[pd.Series, pd.DataFrame] = None,
    var_quantile: Union[float, List] = .05,
    timeframes: Union[None, dict] = None,
    return_tangency_weights: bool = True,
    correlations: Union[bool, List] = True,
    keep_columns: Union[list, str] = None,
    drop_columns: Union[list, str] = None,
    keep_indexes: Union[list, str] = None,
    drop_indexes: Union[list, str] = None,
    drop_before_keep: bool = False,
    _timeframe_name: str = None,
):
    """
    Calculates summary statistics for a time series of returns.

    Parameters:
    returns (pd.DataFrame or List): Time series of returns.
    annual_factor (int, default=None): Factor for annualizing returns.
    provided_excess_returns (bool, default=None): Whether excess returns are already provided.
    rf (pd.Series or pd.DataFrame, default=None): Risk-free rate data.
    var_quantile (float or list, default=0.05): Quantile for Value at Risk (VaR) calculation.
    timeframes (dict or None, default=None): Dictionary of timeframes to calculate statistics for each period.
    return_tangency_weights (bool, default=True): If True, returns tangency portfolio weights.
    correlations (bool or list, default=True): If True, returns correlations, or specify columns for correlations.
    keep_columns (list or str, default=None): Columns to keep in the resulting DataFrame.
    drop_columns (list or str, default=None): Columns to drop from the resulting DataFrame.
    keep_indexes (list or str, default=None): Indexes to keep in the resulting DataFrame.
    drop_indexes (list or str, default=None): Indexes to drop from the resulting DataFrame.
    drop_before_keep (bool, default=False): Whether to drop specified columns/indexes before keeping.

    Returns:
    pd.DataFrame: Summary statistics of the returns.
    """
    returns = returns.copy()
    if isinstance(rf, (pd.Series, pd.DataFrame)):
        rf = rf.copy()
        if provided_excess_returns is True:
            raise Exception(
                'rf is provided but excess returns were provided as well.'
                'Remove "rf" or set "provided_excess_returns" to None or False'
            )
        
    if isinstance(returns, list):
        returns_list = returns[:]
        returns = pd.DataFrame({})
        for series in returns_list:
            returns = returns.merge(series, right_index=True, left_index=True, how='outer')
    """
    This functions returns the summary statistics for the input total/excess returns passed
    into the function
    """
    if 'date' in returns.columns.str.lower():
        returns = returns.rename({'Date': 'date'}, axis=1)
        returns = returns.set_index('date')
    returns.index.name = 'date'

    try:
        returns.index = pd.to_datetime(returns.index.map(lambda x: x.date()))
    except AttributeError:
        print('Could not convert "date" index to datetime.date')
        pass

    returns = returns.apply(lambda x: x.astype(float))

    if annual_factor is None:
        print('Assuming monthly returns with annualization term of 12')
        annual_factor = 12

    if provided_excess_returns is None:
        print(
            'Assuming excess returns were provided to calculate Sharpe.'
            ' If returns were provided (steady of excess returns), the column "Sharpe" is actually "Mean/Volatility"'
        )
        provided_excess_returns = True
    elif provided_excess_returns is False:
        if rf is not None:
            if len(rf.index) != len(returns.index):
                raise Exception('"rf" index must be the same lenght as "returns"')
            print('"rf" is used to subtract returns to calculate Sharpe, but nothing else')

    if isinstance(timeframes, dict):
        all_timeframes_summary_statistics = pd.DataFrame({})
        for name, timeframe in timeframes.items():
            if timeframe[0] and timeframe[1]:
                timeframe_returns = returns.loc[timeframe[0]:timeframe[1]]
            elif timeframe[0]:
                timeframe_returns = returns.loc[timeframe[0]:]
            elif timeframe[1]:
                timeframe_returns = returns.loc[:timeframe[1]]
            else:
                timeframe_returns = returns.copy()
            if len(timeframe_returns.index) == 0:
                raise Exception(f'No returns for {name} timeframe')
            timeframe_returns = timeframe_returns.rename(columns=lambda c: c + f' {name}')
            timeframe_summary_statistics = calc_summary_statistics(
                returns=timeframe_returns,
                annual_factor=annual_factor,
                provided_excess_returns=provided_excess_returns,
                rf=rf,
                var_quantile=var_quantile,
                timeframes=None,
                correlations=correlations,
                _timeframe_name=name,
                keep_columns=keep_columns,
                drop_columns=drop_columns,
                keep_indexes=keep_indexes,
                drop_indexes=drop_indexes,
                drop_before_keep=drop_before_keep
            )
            all_timeframes_summary_statistics = pd.concat(
                [all_timeframes_summary_statistics, timeframe_summary_statistics],
                axis=0
            )
        return all_timeframes_summary_statistics

    summary_statistics = pd.DataFrame(index=returns.columns)
    summary_statistics['Mean'] = returns.mean()
    summary_statistics['Annualized Mean'] = returns.mean() * annual_factor
    summary_statistics['Vol'] = returns.std()
    summary_statistics['Annualized Vol'] = returns.std() * np.sqrt(annual_factor)
    try:
        if not provided_excess_returns:
            if type(rf) == pd.DataFrame:
                rf = rf.iloc[:, 0].to_list()
            elif type(rf) == pd.Series:
                rf = rf.to_list()
            else:
                raise Exception('"rf" must be either a pd.DataFrame or pd.Series')
            excess_returns = returns.apply(lambda x: x - rf)
            summary_statistics['Sharpe'] = excess_returns.mean() / returns.std()
        else:
            summary_statistics['Sharpe'] = returns.mean() / returns.std()
    except Exception as e:
        print(f'Could not calculate Sharpe: {e}')
    summary_statistics['Annualized Sharpe'] = summary_statistics['Sharpe'] * np.sqrt(annual_factor)
    summary_statistics['Min'] = returns.min()
    summary_statistics['Max'] = returns.max()
    summary_statistics['Skewness'] = returns.skew()
    summary_statistics['Excess Kurtosis'] = returns.kurtosis()
    var_quantile = [var_quantile] if isinstance(var_quantile, (float, int)) else var_quantile
    for var_q in var_quantile:
        summary_statistics[f'Historical VaR ({var_q:.2%})'] = returns.quantile(var_q, axis = 0)
        summary_statistics[f'Annualized Historical VaR ({var_q:.2%})'] = returns.quantile(var_q, axis = 0) * np.sqrt(annual_factor)
        summary_statistics[f'Historical CVaR ({var_q:.2%})'] = returns[returns <= returns.quantile(var_q, axis = 0)].mean()
        summary_statistics[f'Annualized Historical CVaR ({var_q:.2%})'] = returns[returns <= returns.quantile(var_q, axis = 0)].mean() * np.sqrt(annual_factor)
    
    wealth_index = 1000 * (1 + returns).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks

    summary_statistics['Max Drawdown'] = drawdowns.min()
    summary_statistics['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_statistics['Bottom'] = drawdowns.idxmin()

    if return_tangency_weights:
        tangency_weights = calc_tangency_weights(returns)
        summary_statistics = summary_statistics.join(tangency_weights)
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_statistics['Recovery'] = recovery_date
    try:
        summary_statistics["Duration (days)"] = [
            (i - j).days if i != "-" else "-" for i, j in
            zip(summary_statistics["Recovery"], summary_statistics["Bottom"])
        ]
    except (AttributeError, TypeError) as e:
        print(f'Cannot calculate "Drawdown Duration" calculation because there was no recovery or because index are not dates: {str(e)}')

    if correlations is True or isinstance(correlations, list):
        returns_corr = returns.corr()
        if _timeframe_name:
            returns_corr = returns_corr.rename(columns=lambda c: c.replace(f' {_timeframe_name}', ''))
        returns_corr = returns_corr.rename(columns=lambda c: c + ' Correlation')
        if isinstance(correlations, list):
            correlation_names = [c + ' Correlation' for c  in correlations]
            not_in_returns_corr = [c for c in correlation_names if c not in returns_corr.columns]
            if len(not_in_returns_corr) > 0:
                not_in_returns_corr = ", ".join([c.replace(' Correlation', '') for c in not_in_returns_corr])
                raise Exception(f'{not_in_returns_corr} not in returns columns')
            returns_corr = returns_corr[[c + ' Correlation' for c  in correlations]]
        summary_statistics = summary_statistics.join(returns_corr)
    
    return filter_columns_and_indexes(
        summary_statistics,
        keep_columns=keep_columns,
        drop_columns=drop_columns,
        keep_indexes=keep_indexes,
        drop_indexes=drop_indexes,
        drop_before_keep=drop_before_keep
    )

def calc_correlations(
    returns: pd.DataFrame,
    print_highest_lowest: bool = True,
    matrix_size: Union[int, float] = 7,
    return_heatmap: bool = True,
    keep_columns: Union[list, str] = None,
    drop_columns: Union[list, str] = None,
    keep_indexes: Union[list, str] = None,
    drop_indexes: Union[list, str] = None,
    drop_before_keep: bool = False
):
    """
    Calculates the correlation matrix of the provided returns and optionally prints or visualizes it.

    Parameters:
    returns (pd.DataFrame): Time series of returns.
    print_highest_lowest (bool, default=True): If True, prints the highest and lowest correlations.
    matrix_size (int or float, default=7): Size of the heatmap for correlation matrix visualization.
    return_heatmap (bool, default=True): If True, returns a heatmap of the correlation matrix.
    keep_columns (list or str, default=None): Columns to keep in the resulting DataFrame.
    drop_columns (list or str, default=None): Columns to drop from the resulting DataFrame.
    keep_indexes (list or str, default=None): Indexes to keep in the resulting DataFrame.
    drop_indexes (list or str, default=None): Indexes to drop from the resulting DataFrame.
    drop_before_keep (bool, default=False): Whether to drop specified columns/indexes before keeping.

    Returns:
    sns.heatmap or pd.DataFrame: Heatmap of the correlation matrix or the correlation matrix itself.
    """
    returns = returns.copy()

    if 'date' in returns.columns.str.lower():
        returns = returns.rename({'Date': 'date'}, axis=1)
        returns = returns.set_index('date')
    returns.index.name = 'date'

    correlation_matrix = returns.corr()
    if return_heatmap:
        fig, ax = plt.subplots(figsize=(matrix_size * 1.5, matrix_size))
        heatmap = sns.heatmap(
            correlation_matrix, 
            xticklabels=correlation_matrix.columns,
            yticklabels=correlation_matrix.columns,
            annot=True,
        )

    if print_highest_lowest:
        highest_lowest_corr = (
            correlation_matrix
            .unstack()
            .sort_values()
            .reset_index()
            .set_axis(['asset_1', 'asset_2', 'corr'], axis=1)
            .loc[lambda df: df.asset_1 != df.asset_2]
        )
        highest_corr = highest_lowest_corr.iloc[lambda df: len(df)-1, :]
        lowest_corr = highest_lowest_corr.iloc[0, :]
        print(f'The highest correlation ({highest_corr["corr"]:.2%}) is between {highest_corr.asset_1} and {highest_corr.asset_2}')
        print(f'The lowest correlation ({lowest_corr["corr"]:.2%}) is between {lowest_corr.asset_1} and {lowest_corr.asset_2}')
    
    if return_heatmap:
        return heatmap
    else:
        return filter_columns_and_indexes(
            correlation_matrix,
            keep_columns=keep_columns,
            drop_columns=drop_columns,
            keep_indexes=keep_indexes,
            drop_indexes=drop_indexes,
            drop_before_keep=drop_before_keep
        )

def calc_cummulative_returns(
    returns: Union[pd.DataFrame, pd.Series],
    return_plot: bool = True,
    fig_size: tuple = (7, 5),
    return_series: bool = False,
    name: str = None,
    timeframes: Union[None, dict] = None,
):
    """
    Calculates cumulative returns from a time series of returns.

    Parameters:
    returns (pd.DataFrame or pd.Series): Time series of returns.
    return_plot (bool, default=True): If True, plots the cumulative returns.
    fig_size (tuple, default=(7, 5)): Size of the plot for cumulative returns.
    return_series (bool, default=False): If True, returns the cumulative returns as a DataFrame.
    name (str, default=None): Name for the title of the plot or the cumulative return series.
    timeframes (dict or None, default=None): Dictionary of timeframes to calculate cumulative returns for each period.

    Returns:
    pd.DataFrame or None: Returns cumulative returns DataFrame if `return_series` is True.
    """
    if timeframes is not None:
        for name, timeframe in timeframes.items():
            if timeframe[0] and timeframe[1]:
                timeframe_returns = returns.loc[timeframe[0]:timeframe[1]]
            elif timeframe[0]:
                timeframe_returns = returns.loc[timeframe[0]:]
            elif timeframe[1]:
                timeframe_returns = returns.loc[:timeframe[1]]
            else:
                timeframe_returns = returns.copy()
            if len(timeframe_returns.index) == 0:
                raise Exception(f'No returns for {name} timeframe')
            calc_cummulative_returns(
                timeframe_returns,
                return_plot=True,
                fig_size=fig_size,
                return_series=False,
                name=name,
                timeframes=None
            )
        return
    returns = returns.copy()
    if isinstance(returns, pd.Series):
        returns = returns.to_frame()
    returns = returns.apply(lambda x: x.astype(float))
    returns = returns.apply(lambda x: x + 1)
    returns = returns.cumprod()
    returns = returns.apply(lambda x: x - 1)
    title = f'Cummulative Returns {name}' if name else 'Cummulative Returns'
    if return_plot:
        returns.plot(
            title=title,
            figsize=fig_size,
            grid=True,
            xlabel='Date',
            ylabel='Cummulative Returns'
        )
    if return_series:
        return returns


#usage to get the cols 
'''
calc_summary_statistics(
    assets_excess_returns[['TIP', 'BWX', 'IEF']],
    annual_factor=12,
    provided_excess_returns=True,
    keep_columns=[
        "Annualized Mean", "Annualized Vol", "Annualized Sharpe",
        "Max Drawdown", "Peak", "Bottom", "Correlation",
        "Annualized Historical VaR", "Kurtosis", "Skewness"
    ]
).transpose()
'''

#Correlations
'''
calc_correlations(assets_excess_returns)
'''

# Cum returns 
'''
calc_cummulative_returns(assets_excess_returns[['TIP', 'BWX', 'IEF']])
'''

# Sharpe V tangency weights

'''
analysis_sharpe_vs_tangent_weights = (
    calc_summary_statistics(
        assets_excess_returns,
        annual_factor=12,
        provided_excess_returns=True,
        keep_columns=['Tangency Weights', 'Annualized Sharpe']
    )
    .sort_values('Annualized Sharpe', ascending=False)
)
analysis_sharpe_vs_tangent_weights
'''

#Tangency weights
'''
calc_tangency_weights(assets_excess_returns, return_graphic=True)

'''


## Allocations - with a mu month target 


# Equal weights - 1/n

'''
n_assets = len(assets_excess_returns.columns)
MU_MONTH_TARGET = 0.010

portfolio_equal_weights_not_scaled = create_portfolio(
    assets_excess_returns,
    weights=[1 / n_assets for _ in range(n_assets)],
    port_name="Equal Weights"
)
portfolio_equal_weights = portfolio_equal_weights_not_scaled * MU_MONTH_TARGET / portfolio_equal_weights_not_scaled.mean()
portfolio_equal_weights
'''


#Risk parity - 1/sigma^2

'''
asset_variance_dict = assets_excess_returns.std().map(lambda x: x ** 2).to_dict()
asset_inv_variance_dict = {asset: 1 / variance for asset, variance in asset_variance_dict.items()}
portfolio_risk_parity_not_scaled = create_portfolio(
    assets_excess_returns,
    weights=asset_inv_variance_dict,
    port_name="Risk Parity"
)
portfolio_risk_parity = portfolio_risk_parity_not_scaled * MU_MONTH_TARGET / portfolio_risk_parity_not_scaled.mean()
portfolio_risk_parity
'''

# Regularised - sigma + diag(sigma) / 2 
'''
portfolio_regularized_not_scaled = calc_tangency_weights(assets_excess_returns, return_port_ret=True, cov_mat=.5, name="Regularized")
portfolio_regularized = portfolio_regularized_not_scaled * MU_MONTH_TARGET / portfolio_regularized_not_scaled.mean()
portfolio_regularized
'''

'\nportfolio_regularized_not_scaled = calc_tangency_weights(assets_excess_returns, return_port_ret=True, cov_mat=.5, name="Regularized")\nportfolio_regularized = portfolio_regularized_not_scaled * MU_MONTH_TARGET / portfolio_regularized_not_scaled.mean()\nportfolio_regularized\n'

* HW2

In [15]:
def stats(data, portfolio = None, portfolio_name = 'Portfolio', annualize = True):
    
    if portfolio is None:
        returns = data
    else:
        returns = data @ portfolio
    
    output = returns.agg(['mean','std'])
    output.loc['sharpe'] = output.loc['mean'] / output.loc['std']
    
    if annualize == True:
        output.loc['mean'] *= 12
        output.loc['std'] *= np.sqrt(12)
        output.loc['sharpe'] *= np.sqrt(12)
    
    if portfolio is None:
        pass
    else:
        output.columns = [portfolio_name]
    
    return output

# Create function to calculate max drawdown and associated dates

def max_drawdown(data, portfolio = None, portfolio_name = 'Portfolio'):
    
    if portfolio is None:
        returns = data
        output = pd.DataFrame(columns=returns.columns)
    else:
        returns = data @ portfolio
        output = pd.DataFrame(columns=[portfolio_name])
    
    cumulative = (returns + 1).cumprod()
    maximum = cumulative.expanding().max()
    drawdown = cumulative / maximum - 1
    
    for col in output.columns:
        
        output.loc['MDD',col] = drawdown[col].min()
        output.loc['Max Date',col] = cumulative[cumulative.index < drawdown[col].idxmin()][col]\
                                             .idxmax()\
                                             .date()
        output.loc['Min Date',col] = drawdown[col].idxmin().date()
        recovery_date = drawdown.loc[drawdown[col].idxmin():,col]\
                                             .apply(lambda x: 0 if x == 0 else np.nan)\
                                             .idxmax()
        
        if pd.isna(recovery_date):
            output.loc['Recovery Date',col] = recovery_date
            output.loc['Recovery Period',col] = np.nan
        else:
            output.loc['Recovery Date',col] = recovery_date.date()
            output.loc['Recovery Period',col] = (output.loc['Recovery Date',col]\
                                             - output.loc['Min Date',col])\
                                             .days
        
    return output

# Create function to retrieve other statistics

def stats_tail_risk(data, portfolio = None, portfolio_name = 'Portfolio', VaR = 0.05):
    
    if portfolio is None:
        returns = data
    else:
        returns = data @ portfolio
    
    output = returns.agg(['skew',
                          'kurt'])
    output.loc['VaR'] = returns.quantile(q = 0.05)
    output.loc['CVaR'] = returns[returns <= output.loc['VaR']].mean()
    output = pd.concat([output, max_drawdown(returns,portfolio,portfolio_name)])
    
    if portfolio is None:
        pass
    else:
        output.columns = portfolio_name
    
    return output

# Create function to display regression stats

def stats_OLS(model,y,x):
    
    output = model.params.to_frame(name = y.columns[0])
    
    return output

import matplotlib.ticker as mtick

def plot_max_drawdown(data, portfolio=None, portfolio_name='Portfolio', ax=None, show_legend=True):
    """
    Plot drawdowns (cumulative peak-to-trough) and annotate the max drawdown window.
    Works with a returns DataFrame/Series, or with a weight vector via `portfolio`.
    """
    # Resolve returns shape and column names
    if portfolio is None:
        returns = data.copy()
        if isinstance(returns, pd.Series):
            returns = returns.to_frame(name=returns.name or portfolio_name)
    else:
        # matrix @ vector -> Series
        returns = (data @ portfolio).to_frame(name=portfolio_name)

    # Compute drawdowns
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.cummax()
    drawdown = cumulative / running_max - 1.0

    # Prep axes
    created_ax = False
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 4))
        created_ax = True

    # Plot each series' drawdown
    for col in drawdown.columns:
        dd = drawdown[col]
        ax.plot(dd.index, dd.values, label=col, linewidth=1.5)

        # Identify MDD stats for annotation/shading
        min_date = dd.idxmin()
        mdd_val = dd.loc[min_date]

        # Peak date = last running max before min (highest cumulative up to min_date)
        peak_date = cumulative.loc[:min_date, col].idxmax()

        # Recovery date = first time drawdown returns to 0 after min_date (if ever)
        post = dd.loc[min_date:]
        recovery_candidates = post[post >= -1e-12]
        recovery_date = recovery_candidates.index[0] if len(recovery_candidates) else None

        # Shade peak -> recovery (or to series end if unrecovered)
        shade_end = recovery_date if recovery_date is not None else dd.index[-1]
        ax.axvspan(peak_date, shade_end, alpha=0.12)

        # Annotate the trough
        ax.scatter([min_date], [mdd_val], zorder=3)
        ax.annotate(
            f"MDD: {mdd_val:.2%}\n{min_date.date()}",
            xy=(min_date, mdd_val),
            xytext=(10, 10),
            textcoords="offset points",
            bbox=dict(boxstyle="round,pad=0.2", fc="white", alpha=0.7),
            arrowprops=dict(arrowstyle="->", lw=1),
        )

    ax.set_title("Drawdown (Peak → Trough → Recovery)")
    ax.set_ylabel("Drawdown")
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.grid(True, linestyle="--", alpha=0.4)
    if show_legend and drawdown.shape[1] > 1:
        ax.legend(frameon=False)
    if created_ax:
        plt.tight_layout()

    return drawdown

# Stats - mean std, sharpe
'''
stats(hf_series) # assumes annualised
'''

# Stats with the tail risk
'''
stats_tail_risk(hf_series) # assumes annualised 
'''


# For generating market beta, treynor and IR
'''
x = sm.add_constant(merrill_factors['SPY US Equity'])

regression_stats = pd.DataFrame(index = ['$B^{SPY}$','Treynor','IR'],columns = hf_series.columns)

for hf in hf_series.columns:
    
    y = hf_series[hf]
    
    model = sm.OLS(y,x).fit()
    
    beta = model.params.iloc[1]
    treynor = 12 * y.mean() / beta
    ir = np.sqrt(12) * model.params.iloc[0] / model.resid.std()
    
    regression_stats[hf] = pd.Series([beta,treynor,ir],index = ['$B^{SPY}$','Treynor','IR'])

regression_stats.round(3)
'''


# OLS stats
'''
y = hf_series[['HFRIFWI Index']]
x = sm.add_constant(merrill_factors)

replication_model = sm.OLS(y,x).fit()

replication_stats = stats_OLS(replication_model,y,x).round(3)
replication_stats

print(f'R-Squared: {round(replication_model.rsquared,2)}')


print(f'The volatility of the tracking error is {round(replication_model.resid.std()*np.sqrt(12),4)}')

'''

# OOS OLS 
'''
t = 60
n = len(hf_series['HFRIFWI Index'])

data = hf_series[['HFRIFWI Index']].copy()
data['Replication'] = np.nan

for i in range(t, n):
    
    y = hf_series['HFRIFWI Index'].iloc[i - 60:i]
    x = sm.add_constant(merrill_factors).iloc[i - 60:i]
    
    m = sm.OLS(y,x).fit()
    
    oos_val = sm.add_constant(merrill_factors).iloc[i].to_numpy().reshape((7))

    rep_val = m.predict(oos_val)
    
    data['Replication'].iloc[i] = rep_val

    
# We can simplify by doing the following:

x = sm.add_constant(merrill_factors)
y = hf_series['HFRIFWI Index']

from statsmodels.regression.rolling import RollingOLS
rolling = RollingOLS(y,x,window=60).fit()
rolling_betas = rolling.params
replication_rolling = (rolling_betas.shift() * x).dropna().sum(axis=1)


oos_loss = (data.dropna().diff(axis=1)**2)['Replication'].sum()
oos_mean = data.dropna()['HFRIFWI Index'].mean()
oos_loss_null = ((data.dropna()['HFRIFWI Index'] - oos_mean)**2).sum()

oos_r2 = 1 - oos_loss / oos_loss_null

print(f'The OOS R-Squared of the replication is {round(oos_r2,4)}')
data.corr().round(3)

'''

#OLS no int 
'''

y = hf_series[['HFRIFWI Index']]
x = merrill_factors

replication_model_no_int = sm.OLS(y,x).fit()

# No-intercept model
stats_OLS(replication_model_no_int,y,x).round(2)

round(replication_model_no_int.predict(x).mean(),4)
round(replication_model.predict(sm.add_constant(x)).mean(),4)

print('Correlation of no intercept model to HFRIFWI: ', np.corrcoef(y['HFRIFWI Index'], replication_model_no_int.predict(x))[0][1])
print('Correlation of intercept model to HFRIFWI: ', np.corrcoef(y['HFRIFWI Index'], replication_model.predict(sm.add_constant(x)))[0][1])
'''

"\n\ny = hf_series[['HFRIFWI Index']]\nx = merrill_factors\n\nreplication_model_no_int = sm.OLS(y,x).fit()\n\n# No-intercept model\nstats_OLS(replication_model_no_int,y,x).round(2)\n\nround(replication_model_no_int.predict(x).mean(),4)\nround(replication_model.predict(sm.add_constant(x)).mean(),4)\n\nprint('Correlation of no intercept model to HFRIFWI: ', np.corrcoef(y['HFRIFWI Index'], replication_model_no_int.predict(x))[0][1])\nprint('Correlation of intercept model to HFRIFWI: ', np.corrcoef(y['HFRIFWI Index'], replication_model.predict(sm.add_constant(x)))[0][1])\n"

* HW3 

In [16]:
# log returns 
'''
def calc_analytics_by_year(data, agg_years):
    res = []
    for y in agg_years:
        sub = data.loc[str(y[0]) : str(y[1])]
        res.append({"mean": sub.mean() * 12, "vol": sub.std() * np.sqrt(12)})
    return pd.DataFrame(res, index=[f"{i[0]} - {i[1]}" for i in agg_years]).stack()


df = pd.read_excel(
    "barnstable_analysis_data.xlsx", sheet_name="data", parse_dates=True, index_col=0
)
df["excess_returns"] = df["SPX"] - df["TB1M"]
df_subsample = df.loc["1965":"1999"]


agg_years = [(1965, 1999), (2000, 2024), (1926, 2024)]

sum_stats = df.apply(calc_analytics_by_year, agg_years=agg_years).T
log_sum_stats = np.log(1 + df).apply(calc_analytics_by_year, agg_years=agg_years).T

res_stats = pd.concat([sum_stats, log_sum_stats])
res_stats.index = pd.MultiIndex.from_product(
    [["levels", "logs"], sum_stats.index.to_list()]
)
res_stats.style.format("{:,.2%}")
'''


#prob underperformance 

'''
def prob_underperformance(mu, sigma, h):
    return norm.cdf(np.sqrt(h) * (-mu / sigma))


mu, sigma = res_stats["1965 - 1999"].loc[("logs", "excess_returns")]

print(
    f"SPX underperforming risk-free rate in the 15 years after 1999: {prob_underperformance(mu=mu, sigma=sigma, h=15):,.2%}"
)
print(
    f"SPX underperforming risk-free rate in the 30 years after 1999: {prob_underperformance(mu=mu, sigma=sigma, h=30):,.2%}"
)

h = np.linspace(0, 30, 300)

probabilities = prob_underperformance(mu=mu, sigma=sigma, h=h)


plt.plot(h, probabilities)
plt.title(
    "Probability of SPX Underperforming Risk-Free Rate\nas a Function of Investment Horizon"
)
plt.xlabel("Investment Horizon (Years)")
plt.ylabel("Probability of Underperformance")
plt.show()
'''

#Full sample 

''' 
# 1965-2024 period
mu = np.log(1 + df.loc["1965":"2023"]).mean() * 12
sigma = np.log(1 + df.loc["1965":"2023"]).std() * np.sqrt(12)
h = np.arange(30) + 1

mu_excess = mu["excess_returns"]
sigma_excess = sigma["excess_returns"]

probabilities_full_sample = prob_underperformance(mu=mu_excess, sigma=sigma_excess, h=h)
print(
    f"SPX underperforming risk-free rate in the 30 years after 2024: {prob_underperformance(mu=mu_excess, sigma=sigma_excess, h=30):,.2%}"
)
'''


#in sample of out of sample likelihood

''' 
mu = (
    res_stats[("1965 - 1999", "mean")].loc[("logs", "SPX")]
    - res_stats[("2000 - 2024", "mean")].loc[("logs", "SPX")]
)
sigma = res_stats[("1965 - 1999", "vol")].loc[("logs", "SPX")]

print(
    f"Probability of underperformance in 2000-2024: {prob_underperformance(mu, sigma, 24):,.2%}"
)
'''

' \nmu = (\n    res_stats[("1965 - 1999", "mean")].loc[("logs", "SPX")]\n    - res_stats[("2000 - 2024", "mean")].loc[("logs", "SPX")]\n)\nsigma = res_stats[("1965 - 1999", "vol")].loc[("logs", "SPX")]\n\nprint(\n    f"Probability of underperformance in 2000-2024: {prob_underperformance(mu, sigma, 24):,.2%}"\n)\n'

## Solutions

# 1.1

# 1.2

# 1.3

# 1.4

# 1.5

# 2.1

# 2.2

# 2.3

# 2.4

# 2.5

# 3.1

# 3.2

# 3.3

# 3.4

# 3.5